In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import threading
import time

class CCTVDetectionSystem:
    def __init__(self, video_source=0, resize_width=640, resize_height=480):
        self.video_source = video_source
        self.cap = None
        self.model = None
        self.resize_width = resize_width
        self.resize_height = resize_height
        
        self.setup_video()
        self.load_model()
        
        self.running = False
        self.people_frame = None
        self.zebra_frame = None
        self.road_frame = None
        self.violation_frame = None
        self.original_frame = None
        
    
    def setup_video(self):
        self.cap = cv2.VideoCapture(self.video_source)
        if not self.cap.isOpened():
            raise ValueError(f"Error: Could not open video source {self.video_source}")
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
    
    def load_model(self):
        try:
            self.model = YOLO('yolov8n.pt')
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            print("Please install ultralytics: pip install ultralytics")
    
    def detect_people(self, frame, results=None):
        if self.model is None:
            return frame
        try:
            annotated_frame = frame.copy()
            if results is None:
                results = self.model(frame, classes=[0])
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                        confidence = box.conf[0].cpu().numpy()
                        if confidence > 0.5:
                            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                            label = f'Person: {confidence:.2f}'
                            cv2.putText(annotated_frame, label, (x1, y1-10), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            return annotated_frame
        except Exception as e:
            print(f"Error in people detection: {e}")
            return frame
    
    def detect_zebra_crossing(self, frame):
        try:
            result_frame = frame.copy()
            height, width = frame.shape[:2]
            
            zebra_polygon = np.array([
                [int(width * 0.825), int(height * 0.460)],
                [int(width * 0.489), int(height * 0.404)],
                [int(width * 0.567), int(height * 0.365)],
                [int(width * 0.864), int(height * 0.404)]
            ], np.int32)
            
            overlay = result_frame.copy()
            cv2.fillPoly(overlay, [zebra_polygon], (0, 255, 255))
            result_frame = cv2.addWeighted(result_frame, 0.7, overlay, 0.3, 0)
            cv2.polylines(result_frame, [zebra_polygon], True, (0, 255, 255), 3)
            
            area = cv2.contourArea(zebra_polygon)
            cv2.putText(result_frame, f'Area: {int(area)} pixels', 
                       (zebra_polygon[0][0], zebra_polygon[3][1] + 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            
            for point in zebra_polygon:
                cv2.circle(result_frame, tuple(point), 5, (255, 0, 0), -1)
            
            cv2.putText(result_frame, 'STATUS: ZEBRA CROSSING ACTIVE', 
                       (10, height - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(result_frame, 'Mode: Predefined Area Detection', 
                       (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            return result_frame
        except Exception as e:
            print(f"Error in zebra crossing detection: {e}")
            return frame
    
    def detect_road_area(self, frame):
        try:
            result_frame = frame.copy()
            height, width = frame.shape[:2]
            
            road_polygon = np.array([
                [int(width * 0.745), int(height * 0.275)],
                [int(width * 0.909), int(height * 0.308)],
                [int(width * 0.514), int(height * 0.990)],
                [int(width * 0.000), int(height * 0.992)],
                [int(width * 0.003), int(height * 0.660)]
            ], np.int32)
            
            overlay = result_frame.copy()
            cv2.fillPoly(overlay, [road_polygon], (255, 150, 50))
            result_frame = cv2.addWeighted(result_frame, 0.6, overlay, 0.4, 0)
            cv2.polylines(result_frame, [road_polygon], True, (255, 100, 0), 3)
            
            for i, point in enumerate(road_polygon):
                cv2.circle(result_frame, tuple(point), 8, (0, 0, 255), -1)
                cv2.putText(result_frame, str(i+1), (point[0]+10, point[1]-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.putText(result_frame, 'ROAD AREA DETECTION', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 100, 0), 2)
            cv2.putText(result_frame, 'PREDEFINED ROAD ZONE', 
                       (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 150, 50), 2)
            
            area = cv2.contourArea(road_polygon)
            cv2.putText(result_frame, f'Road Area: {int(area)} pixels', 
                       (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 150, 50), 2)
            
            frame_area = width * height
            coverage_percent = (area / frame_area) * 100
            cv2.putText(result_frame, f'Coverage: {coverage_percent:.1f}% of frame', 
                       (10, 105), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 150, 50), 2)
            
            cv2.putText(result_frame, 'Road Coordinates (% of frame):', 
                       (10, height - 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            coord_text = [
                "P1: (74.5%, 27.5%)", "P2: (90.9%, 30.8%)", "P3: (51.4%, 99.0%)",
                "P4: (0.0%, 99.2%)", "P5: (0.3%, 66.0%)"
            ]
            for i, text in enumerate(coord_text):
                cv2.putText(result_frame, text, (10, height - 95 + i*15), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
            
            cv2.putText(result_frame, 'STATUS: ROAD AREA ACTIVE', 
                       (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
            return result_frame
        except Exception as e:
            print(f"Error in road area detection: {e}")
            return frame
    
    def detect_violations(self, frame, results=None):
        try:
            # Start with clean frame copy - NO overlays
            result_frame = frame.copy()
            height, width = frame.shape[:2]
            
            zebra_polygon = np.array([
                [int(width * 0.825), int(height * 0.460)],
                [int(width * 0.489), int(height * 0.404)],
                [int(width * 0.567), int(height * 0.365)],
                [int(width * 0.864), int(height * 0.404)]
            ], np.int32)
            
            road_polygon = np.array([
                [int(width * 0.745), int(height * 0.275)],
                [int(width * 0.909), int(height * 0.308)],
                [int(width * 0.514), int(height * 0.990)],
                [int(width * 0.000), int(height * 0.992)],
                [int(width * 0.003), int(height * 0.660)]
            ], np.int32)
            
            if results is None:
                results = self.model(frame, classes=[0])
            
            violation_count = 0
            
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                        confidence = box.conf[0].cpu().numpy()
                        
                        if confidence > 0.2:
                            # Use bottom center of the bounding box
                            center_x = float((x1 + x2) / 2.0)
                            center_y = float(y2)  # Bottom of the box
                            
                            in_road = cv2.pointPolygonTest(road_polygon, (center_x, center_y), False) >= 0
                            in_zebra = cv2.pointPolygonTest(zebra_polygon, (center_x, center_y), False) >= 0
                            
                            # Debug output
                            print(f"Person at bottom center ({center_x}, {center_y}) - in_road: {in_road}, in_zebra: {in_zebra}")
                            
                            # VIOLATION: Person is in road BUT NOT in zebra crossing
                            if in_road and not in_zebra:
                                violation_count += 1
                                cv2.rectangle(result_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                                cv2.putText(result_frame, f'VIOLATION: {confidence:.2f}', (x1, y1-10), 
                                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                                cv2.circle(result_frame, (int(center_x), int(center_y)), 5, (0, 0, 255), -1)
            
            # Display violation count and info - clean display without zone overlays
            cv2.putText(result_frame, f'VIOLATIONS: {violation_count}', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            cv2.putText(result_frame, 'VIOLATION DETECTION - CLEAN VIEW', (10, 55), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(result_frame, 'Red boxes: People in road but NOT in zebra crossing', (10, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # NO zone overlays - just the clean frame with violation markers
            
            return result_frame
            
        except Exception as e:
            print(f"Error in violation detection: {e}")
            return frame
    
    def process_frame(self, frame):
        resized_frame = cv2.resize(frame, (self.resize_width, self.resize_height))
        self.original_frame = resized_frame.copy()
        results = self.model(resized_frame, classes=[0]) if self.model else None
        self.people_frame = self.detect_people(resized_frame, results)
        self.zebra_frame = self.detect_zebra_crossing(resized_frame)
        self.road_frame = self.detect_road_area(resized_frame)
        self.violation_frame = self.detect_violations(resized_frame, results)
    
    def display_windows(self):
        cv2.namedWindow('People Detection') 
        cv2.namedWindow('Zebra Crossing Detection')
        cv2.namedWindow('Road Area Detection')
        cv2.namedWindow('Violation Detection')
        
        while self.running:
            if self.people_frame is not None:
                cv2.imshow('People Detection', self.people_frame)
                
            if self.zebra_frame is not None:
                cv2.imshow('Zebra Crossing Detection', self.zebra_frame)
            
            if self.road_frame is not None:
                cv2.imshow('Road Area Detection', self.road_frame)
            
            if self.violation_frame is not None:
                cv2.imshow('Violation Detection', self.violation_frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                self.running = False
                break
                
        cv2.destroyAllWindows()
    
    def run(self):
        print("Starting CCTV Detection System with Clean Violation Detection...")
        print("="*70)
        print("CONTROLS:")
        print("- Press 'q' to quit")
        print("="*70)
        print("\nDETECTION WINDOWS:")
        print("1. People Detection - Green boxes around detected people")
        print("2. Zebra Crossing - Cyan area marking the crossing zone")
        print("3. Road Area - Orange area marking the road surface")
        print("4. Violation Detection - CLEAN VIEW with RED boxes for violations only")
        print("   (No zone overlays - just violations highlighted)")
        print("="*70 + "\n")
        
        self.running = True
        display_thread = threading.Thread(target=self.display_windows)
        display_thread.daemon = True
        display_thread.start()
        
        start_time = time.time()
        frame_count = 0
        
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                print("Error reading frame or end of video")
                break
            self.process_frame(frame)
            frame_count += 1
            if time.time() - start_time > 1:
                fps = frame_count / (time.time() - start_time)
                print(f"FPS: {fps:.2f}")
                frame_count = 0
                start_time = time.time()
            time.sleep(0.03)
        
        self.cap.release()
        cv2.destroyAllWindows()
        print("CCTV Detection System stopped")

def main():
    video_source = "Video_2025_08_07-1.mp4"  # Replace with full path if needed
    resize_width = 640
    resize_height = 480
    try:
        detector = CCTVDetectionSystem(video_source, resize_width, resize_height)
        detector.run()
    except KeyboardInterrupt:
        print("\nInterrupted by user")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Starting CCTV Detection System with Clean Violation Detection...
CONTROLS:
- Press 'q' to quit

DETECTION WINDOWS:
1. People Detection - Green boxes around detected people
2. Zebra Crossing - Cyan area marking the crossing zone
3. Road Area - Orange area marking the road surface
4. Violation Detection - CLEAN VIEW with RED boxes for violations only
   (No zone overlays - just violations highlighted)


0: 480x640 10 persons, 215.6ms
Speed: 9.2ms preprocess, 215.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Person at bottom center (23.0, 281.0) - in_road: False, in_zebra: False
Person at bottom center (582.5, 309.0) - in_road: False, in_zebra: False
Person at bottom center (538.0, 447.0) - in_road: False, in_zebra: False
Person at bottom center (581.5, 448.0) - in_road: False, in_zebra: False
Person at bottom center (460.5, 444.0) - in_road: False, in_zebra: False
Person at bottom center (323.5, 169.0) - in_road: False, in_zebra: False
Person at bottom center (6.5,